In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from collections import deque
import random
import numpy as np
import pandas as pd

# Hyperparameters for Deep Q-Learning
discount_factor = 0.99
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.995
learning_rate = 0.001
batch_size = 64
memory_size = 2000
num_episodes = 10

# State and action space
state_size = 5  # [sales, price, discount, inventory level, event_type]
action_size_bau = 3  # Actions for BAU [0%, 5%, 10% discount]
action_size_event = 5  # Actions for Event [0%, 5%, 10%, 15%, 20% discount]

# Q-Network model
def build_model(state_size, action_size):
    model = tf.keras.Sequential()
    model.add(layers.Input(shape=(state_size,)))
    model.add(layers.Dense(24, activation='relu'))
    model.add(layers.Dense(24, activation='relu'))
    model.add(layers.Dense(action_size, activation='linear'))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mse')
    return model

# Separate Q-networks for BAU and Event strategies
class DQNAgent:
    def __init__(self, state_size, action_size_bau, action_size_event):
        self.state_size = state_size
        self.action_size_bau = action_size_bau
        self.action_size_event = action_size_event
        self.memory = deque(maxlen=memory_size)
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay

        # Two separate models for BAU and event days
        self.model_bau = build_model(state_size, action_size_bau)
        self.model_event = build_model(state_size, action_size_event)

    # Modify remember function to store event_type
    def remember(self, state, action, reward, next_state, done, event_type):
        self.memory.append((state, action, reward, next_state, done, event_type))

    # Action selection based on event type
    def act(self, state, event_type):
        if np.random.rand() <= self.epsilon:
            if event_type == 'BAU':
                return random.randrange(self.action_size_bau)  # Explore BAU actions
            else:
                return random.randrange(self.action_size_event)  # Explore Event actions
        else:
            if event_type == 'BAU':
                q_values = self.model_bau.predict(state)
                return np.argmax(q_values[0])  # Exploit BAU Q-values
            else:
                q_values = self.model_event.predict(state)
                return np.argmax(q_values[0])  # Exploit Event Q-values

    # Replay function to replay experience and train the model
    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done, event_type in minibatch:
            target = reward
            if not done:
                if event_type == 'BAU':
                    target = reward + discount_factor * np.amax(self.model_bau.predict(next_state)[0])
                    target_f = self.model_bau.predict(state)
                    target_f[0][action] = target
                    self.model_bau.fit(state, target_f, epochs=1, verbose=0)
                else:
                    target = reward + discount_factor * np.amax(self.model_event.predict(next_state)[0])
                    target_f = self.model_event.predict(state)
                    target_f[0][action] = target
                    self.model_event.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# Initialize the DQN Agent with separate action spaces for BAU and Events
agent = DQNAgent(state_size, action_size_bau, action_size_event)

# Load the dataset (replace with the correct path to your dataset)
df = pd.read_csv(r"/content/synthetic_sales_data.csv")

# Function to simulate an episode (pricing decisions over a period)
def run_episode(df):
    total_reward = 0
    for i in range(len(df)):
        # State includes Sales, Price, Discount, Inventory, and Event Type
        state = np.array([df['Sales'][i], df['Price'][i], df['Discount_Rate'][i], 100, 0]).reshape(1, -1)
        event_type = df['Event_Type'][i]

        # Act based on the type of day (BAU or Event)
        action = agent.act(state, event_type)

        # Take action and calculate reward
        next_state = np.array([df['Sales'][i], df['Price'][i], df['Discount_Rate'][i], 100, 0]).reshape(1, -1)
        reward = df['Revenue'][i]
        done = i == len(df) - 1

        # Remember for experience replay (now includes event_type)
        agent.remember(state, action, reward, next_state, done, event_type)
        total_reward += reward

        if done:
            print(f"Episode finished with total reward: {total_reward}")
            break
    return total_reward

# Train the agent over multiple episodes
for e in range(num_episodes):
    total_reward = run_episode(df)
    print(f"Episode {e+1}/{num_episodes}, Total Reward: {total_reward}")
    agent.replay(batch_size)


Streaming output truncated to the last 5000 lines.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step